In [1]:
import os
from pathlib import Path
import pandas as pd
import QuantLib as ql
import numpy as np
from datetime import datetime
from model_settings import ms
from itertools import product
file_dir = os.path.join(str(Path().resolve()),'historical_data_collection','historical_av_collection')
files = os.listdir(file_dir)
filename = [f for f in files if f.endswith('.h5')][0]

file = os.path.join(file_dir,filename)
with pd.HDFStore(file) as store:
    df = store["/date_2022_10_20/heston_calibration/calibration_results"]
eg = df.iloc[0]
eg


pricing settings:
Actual/365 (Fixed) day counter
New York stock exchange calendar
compounding: continuous
frequency: annual



strike_price             335.0
w                         call
market_price              34.6
volatility             0.33141
days_to_maturity            34
calculation_date    2022-10-20
spot_price              365.41
risk_free_rate            0.04
dividend_rate              0.0
theta                  0.17771
kappa                 0.412367
eta                   0.785592
rho                  -0.582856
v0                     0.08079
moneyness             0.090776
black_scholes        35.227613
heston_price          34.77996
error                -0.012707
Name: 3420, dtype: object

In [2]:
past_fixings = 0
rng = "pseudorandom" # could use "lowdiscrepancy"
numPaths = 100000


def asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,kappa,theta,rho,eta,v0,calculation_datetime):
    if w == 'call':
        option_type = ql.Option.Call 
    elif w == 'put':
        option_type = ql.Option.Put
    t = n_fixings*fixing_frequency
    # if isinstance(calculation_datetime, np.datetime64):
    #     calculation_datetime = calculation_datetime.astype('M8[ms]').astype(datetime)
    
    calculation_date = ql.Date(calculation_datetime.day,calculation_datetime.month,calculation_datetime.year)
    
    periods = np.arange(fixing_frequency,t+1,fixing_frequency).astype(int)
    
    fixing_dates = [calculation_date + ql.Period(int(p),ql.Days) for p in periods]
    expiration_date = calculation_date + ql.Period(int(t),ql.Days)
    
    riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, float(r), ql.Actual365Fixed()))
    dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, float(g), ql.Actual365Fixed()))
    
    hestonProcess = ql.HestonProcess(riskFreeTS, dividendTS, ql.QuoteHandle(ql.SimpleQuote(s)), v0, kappa, theta, eta, rho)
    hestonModel = ql.HestonModel(hestonProcess)
    vanillaPayoff = ql.PlainVanillaPayoff(option_type, float(k))
    europeanExercise = ql.EuropeanExercise(expiration_date)
    
    if averaging_type == 'geometric':
        geometric_engine = ql.MCDiscreteGeometricAPHestonEngine(hestonProcess, rng, requiredSamples=numPaths,seed=123)
        geometricAverage = ql.Average().Geometric
        geometricRunningAccumulator = 1.0
        discreteGeometricAsianOption = ql.DiscreteAveragingAsianOption(
        	geometricAverage, geometricRunningAccumulator, past_fixings,
            fixing_dates, vanillaPayoff, europeanExercise)
        discreteGeometricAsianOption.setPricingEngine(geometric_engine)
        return discreteGeometricAsianOption.NPV()
        
    elif averaging_type == 'arithmetic':
        arithmetic_engine = ql.MCDiscreteArithmeticAPHestonEngine(hestonProcess, rng, requiredSamples=numPaths)
        arithmeticAverage = ql.Average().Arithmetic
        arithmeticRunningAccumulator = 0.0
        discreteArithmeticAsianOption = ql.DiscreteAveragingAsianOption(
            arithmeticAverage, arithmeticRunningAccumulator, past_fixings, 
            fixing_dates, vanillaPayoff, europeanExercise)
        discreteArithmeticAsianOption.setPricingEngine(arithmetic_engine)
        return discreteArithmeticAsianOption.NPV()
    else:
        print("invalid Asian option averaging type out of 'arithmetic' and geometric'")
        pass

In [3]:
vector_asian_price = np.vectorize(asian_option_price)
calculation_datetime = datetime.today()

r = 0.04
g = 0.0

w = 'put'


s = 365.41
k = 370.00
kappa = 0.412367
theta = 0.17771
rho = -0.582856
eta = 0.785592
v0 = 0.08079
n_fixings = 5
fixing_frequency = 50

averaging_type = 'geometric'

t = n_fixings*fixing_frequency


price = asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,kappa,theta,rho,eta,v0,calculation_datetime)

vanilla = ms.ql_heston_price(s,k,t,r,g,w,kappa,theta,rho,eta,v0,datetime.today())

print(f"{price}\n{vanilla}")

21.715962339603674
28.766454289828687


In [4]:
vector_asian_price = np.vectorize(asian_option_price)

# prices = vector_asian_price(s,[300,370,405],r,g,w,averaging_type,n_fixings,fixing_frequency,kappa,theta,rho,eta,v0,calculation_date)

In [5]:
fixing_frequency = [1,7,30,90]
n_fixings = [1,2,5,10]
K = np.unique(np.linspace(s*0.5,s*1.5,10).astype(int))

features = pd.DataFrame(
    product(
        [s],
        K,
        [r],
        [g],
        fixing_frequency,
        n_fixings,
        ['call','put'],
        ['geometric','arithmetic'],
        [theta],
        [kappa],
        [rho],
        [eta],
        [v0],
        [calculation_datetime]
    ),
    columns = [
        'spot_price','strike_price',
        'risk_free_rate','dividend_rate',
        'fixing_frequency','n_fixings','w','averaging_type',
        'theta','kappa','rho','eta','v0','calculation_date'
    ]
)
features

,spot_price,strike_price,risk_free_rate,dividend_rate,fixing_frequency,n_fixings,w,averaging_type,theta,kappa,rho,eta,v0,calculation_date
0,365.41,182,0.04,0.0,1,1,call,geometric,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 21:32:21.309307
1,365.41,182,0.04,0.0,1,1,call,arithmetic,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 21:32:21.309307
2,365.41,182,0.04,0.0,1,1,put,geometric,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 21:32:21.309307
3,365.41,182,0.04,0.0,1,1,put,arithmetic,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 21:32:21.309307
4,365.41,182,0.04,0.0,1,2,call,geometric,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 21:32:21.309307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,365.41,548,0.04,0.0,90,5,put,arithmetic,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 21:32:21.309307
636,365.41,548,0.04,0.0,90,10,call,geometric,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 21:32:21.309307
637,365.41,548,0.04,0.0,90,10,call,arithmetic,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 21:32:21.309307
638,365.41,548,0.04,0.0,90,10,put,geometric,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 21:32:21.309307


In [ ]:
features.loc[:,'asian_price'] = vector_asian_price(
    features['spot_price'],
    features['strike_price'],
    features['risk_free_rate'],
    features['dividend_rate'],
    features['w'],
    features['averaging_type'],
    features['n_fixings'],
    features['fixing_frequency'],
    kappa,theta,rho,eta,v0,
    features['calculation_date']
)

In [ ]:
features